In [1]:
# First up, I'll import every library that will be used in this project is imported at the start.

# Data handling and processing
import pandas as pd
import numpy as np

# Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Statistics
from scipy import stats
import statsmodels.api as sm
from scipy.stats import randint as sp_randint
from time import time

# NLP
import nltk
nltk.download('wordnet')
import re
from textblob import TextBlob
from nltk.corpus import stopwords
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

/opt/conda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


[nltk_data] Error loading wordnet: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Reading in data
data = pd.read_csv('../input/nycomments/NYcomments.csv',sep=";",encoding="utf-8")
data = data[['Hotel Names','Sentiment','Comments']]
data.Sentiment=data.Sentiment.astype(str)
data.columns = ['Hotel Names', 'Sentiment', 'Comments']

In [3]:
# Inspecting the variables
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245545 entries, 0 to 245544
Data columns (total 3 columns):
Hotel Names    245545 non-null object
Sentiment      245545 non-null object
Comments       245456 non-null object
dtypes: object(3)
memory usage: 5.6+ MB


,Hotel Names,Sentiment,Comments
0,Hyatt Place New York City/Times Square,1,perfect location! Very friendly staff that hel...
1,Hyatt Place New York City/Times Square,1,Clean and comfortable rooms
2,Hyatt Place New York City/Times Square,0,The best place to be
3,Hyatt Place New York City/Times Square,1,"excellent location, very comfortable rooms and..."
4,Hyatt Place New York City/Times Square,1,Service
5,Hyatt Place New York City/Times Square,1,I got to the hotel 1 hour and 30 minutes early...
6,Hyatt Place New York City/Times Square,1,Esta muy cerca del Time Square. Las habitacion...
7,Hyatt Place New York City/Times Square,1,Sad I could not eat most things. I have a dair...
8,Hyatt Place New York City/Times Square,1,"Location was great, beds were super comfy. Bat..."
9,Hyatt Place New York City/Times Square,1,Great location! Great customer service


In [4]:
# Replacing blank variables with 'unknown' ready for processing
data['Comments'].fillna('unknown', inplace=True)

## 1. Text Preprocessing

Text is the most unstructured form of all the available data, therefore various types of noise are present in it. This means that the data is not readily analysable without any pre-processing. The entire process of cleaning and standardization of text, making it noise-free and ready for analysis is known as text preprocessing. This usually comprises two key steps:
<br>
1. Noise Removal
2. Lexicon Normalisation

### Noise Removal

Any piece of text which is not relevant to the context of the data and the end-output can be specified as the noise.

For example – language stopwords (commonly used words of a language – is, am, the, of, in etc), URLs or links, punctuations and industry specific words. This step deals with removal of all types of noisy entities present in the text.

Following is a python function to strip out noise throughout the reviews:

In [5]:
# Importing SKLearn's list of stopwords and then appending with my own words 
stop = text.ENGLISH_STOP_WORDS

# Basic text cleaning function
def remove_noise(text):
    
    # Make lowercase
    text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    # Remove whitespaces
    text = text.apply(lambda x: " ".join(x.strip() for x in x.split()))
    
    # Remove special characters
    text = text.apply(lambda x: "".join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
    
    # Remove punctuation
    text = text.str.replace('[^\w\s]', '')
    
    # Remove numbers
    text = text.str.replace('\d+', '')
    
    # Remove Stopwords
    text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    # Convert to string
    text = text.astype(str)
        
    return text

In [6]:
# Applying noise removal function to data
data['Filtered Review Text'] = remove_noise(data['Comments'])
data.head()

,Hotel Names,Sentiment,Comments,Filtered Review Text
0,Hyatt Place New York City/Times Square,1,perfect location! Very friendly staff that hel...,perfect location friendly staff helped solve p...
1,Hyatt Place New York City/Times Square,1,Clean and comfortable rooms,clean comfortable rooms
2,Hyatt Place New York City/Times Square,0,The best place to be,best place
3,Hyatt Place New York City/Times Square,1,"excellent location, very comfortable rooms and...",excellent location comfortable rooms close tim...
4,Hyatt Place New York City/Times Square,1,Service,service


Before moving onto lexicon normalisation, I want to gain a sense of the sentiment per review. I don't intend to use this for any machine learning purposes, more-so out of interest to understand whether reviews lean towards positivity or negativity. I'll come back to this again later.

In [7]:
# Defining a sentiment analyser function
def sentiment_analyser(text):
    return text.apply(lambda Text: pd.Series(TextBlob(Text).sentiment.polarity))

# Applying function to reviews
data['Polarity'] = sentiment_analyser(data['Filtered Review Text'])
data.head(10)

,Hotel Names,Sentiment,Comments,Filtered Review Text,Polarity
0,Hyatt Place New York City/Times Square,1,perfect location! Very friendly staff that hel...,perfect location friendly staff helped solve p...,0.368056
1,Hyatt Place New York City/Times Square,1,Clean and comfortable rooms,clean comfortable rooms,0.383333
2,Hyatt Place New York City/Times Square,0,The best place to be,best place,1.000000
3,Hyatt Place New York City/Times Square,1,"excellent location, very comfortable rooms and...",excellent location comfortable rooms close tim...,0.700000
4,Hyatt Place New York City/Times Square,1,Service,service,0.000000
5,Hyatt Place New York City/Times Square,1,I got to the hotel 1 hour and 30 minutes early...,got hotel hour minutes early fault check pm wa...,0.083333
6,Hyatt Place New York City/Times Square,1,Esta muy cerca del Time Square. Las habitacion...,esta muy cerca del time square las habitacione...,0.000000
7,Hyatt Place New York City/Times Square,1,Sad I could not eat most things. I have a dair...,sad eat things dairy allergy dont eat sweets,-0.500000
8,Hyatt Place New York City/Times Square,1,"Location was great, beds were super comfy. Bat...",location great beds super comfy bathroom nice ...,0.560000
9,Hyatt Place New York City/Times Square,1,Great location! Great customer service,great location great customer service,0.800000


### Lexicon Normalisation

Another type of textual noise is about the multiple representations exhibited by single word.

For example – “play”, “player”, “played”, “plays” and “playing” are the different variations of the word – “play”. Though they mean different things, contextually they all are similar. This step converts all the disparities of a word into their normalized form (also known as lemma). Normalization is a pivotal step for feature engineering with text as it converts the high dimensional features (N different features) to the low dimensional space (1 feature), which is an ideal ask for any ML model.

There are two methods of lexicon normalisation; Stemming or Lemmatization. I will opt for Lemmatization, as this will return the root form of each word (rather than just stripping suffixes, which is stemming).

In [8]:
# Instantiate the Word tokenizer & Word lemmatizer
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

# Define a word lemmatizer function
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

# Apply the word lemmatizer function to data
data['Filtered Review Text'] = data['Filtered Review Text'].apply(lemmatize_text)
data.head()

,Hotel Names,Sentiment,Comments,Filtered Review Text,Polarity
0,Hyatt Place New York City/Times Square,1,perfect location! Very friendly staff that hel...,"[perfect, location, friendly, staff, helped, s...",0.368056
1,Hyatt Place New York City/Times Square,1,Clean and comfortable rooms,"[clean, comfortable, room]",0.383333
2,Hyatt Place New York City/Times Square,0,The best place to be,"[best, place]",1.000000
3,Hyatt Place New York City/Times Square,1,"excellent location, very comfortable rooms and...","[excellent, location, comfortable, room, close...",0.700000
4,Hyatt Place New York City/Times Square,1,Service,[service],0.000000


# 2. Getting a text matrix

To analyse a preprocessed data, it needs to be converted into features. Depending upon the usage, text features can be constructed using a variety of techniques – in this kernel I will be converting the data into statistical features.

The specific model in question is known as <b>'Term Frequency – Inverse Document Frequency' (TF – IDF)</b>

TF-IDF is a weighted model commonly used for information retrieval problems. It aims to convert the text documents into vector models on the basis of occurrence of words in the documents without taking considering the exact ordering. For Example – let say there is a dataset of N text documents, In any document “D”, TF and IDF will be defined as –

- <b>Term Frequency (TF)</b> – TF for a term “t” is defined as the count of a term “t” in a document “D”
- <b>Inverse Document Frequency (IDF)</b> – IDF for a term is defined as logarithm of ratio of total documents available in the corpus and number of documents containing the term T.
- <b>TF . IDF</b> – TF IDF formula gives the relative importance of a term in a corpus (list of documents), given by the following formula below. Following is the code using python’s scikit learn package to convert a text into tf idf vectors:

Scikit-learn provides two methods to get to our end result (a TD-IDF weight matrix). One is a two-part process of using the CountVectorizer class to count how many times each term shows up in each document, followed by the TfidfTransformer class generating the weight matrix. The other does both steps in a single TfidfVectorizer class. In this Kernel I will proceed with method one; below is step one:

In [9]:
# Getting a count of words from the documents
# Ngram_range is set to 1,2 - meaning either single or two word combination will be extracted
cvec = CountVectorizer(min_df=.015, max_df=.9, ngram_range=(1,2), tokenizer=lambda doc: doc, lowercase=False)
cvec.fit(data['Filtered Review Text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=0.9, max_features=None, min_df=0.015,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function <lambda> at 0x7f97e071dd08>, vocabulary=None)

In [10]:
# Getting the total n-gram count
len(cvec.vocabulary_)

115

I am happy with that number as a starting point, less than 1000 was my initial aim. If I wanted to be more or less restrictive on n-gram selection, I could adjust the 'min_df' and 'max_df' parameters within my CountVectorizer, which controls for the minimum and maximum amount of documents each word should feature in.

We can now tackle the next step, which is to turn this document into a <b>'bag of words' representation</b>. This creates a separate column for each term that contains the count within each document. After that, we’ll take a look at the <b>sparsity</b> of this representation which lets us know how many <b>nonzero values</b> there are in the dataset. The more sparse the data is the more challenging it will be to model, but that’s a discussion for another day:

In [11]:
# Creating the bag-of-words representation
cvec_counts = cvec.transform(data['Filtered Review Text'])
print('sparse matrix shape:', cvec_counts.shape)
print('nonzero count:', cvec_counts.nnz)
print('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))

sparse matrix shape: (245545, 115)
nonzero count: 1100706
sparsity: 3.90%


Now that we have term counts for each document, the TfidfTransformer can be applied to calculate the weights for each term in each document:

In [12]:
# Instantiating the TfidfTransformer
transformer = TfidfTransformer()

# Fitting and transforming n-grams
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

<245545x115 sparse matrix of type '<class 'numpy.float64'>'
	with 1100706 stored elements in Compressed Sparse Row format>

Great, we have our weighted words! Just a few more steps required (below); I'm going to extract all of the feature names (which are the n-grams) and put these into a DataFrame along with the corresponding weights per review. Then I am going to add in at the end some summary statistics to understand per review:

- The highest weighted word
- The weight of this word
- The total weighting per review.

In [13]:
# Getting a list of all n-grams
transformed_weights = transformed_weights.toarray()
vocab = cvec.get_feature_names()

# Putting weighted n-grams into a DataFrame and computing some summary statistics
model = pd.DataFrame(transformed_weights, columns=vocab)
#model['Keyword'] = model.idxmax(axis=1)
#model['Max'] = model.max(axis=1)
#model['Sum'] = model.drop('Max', axis=1).sum(axis=1)
model.head(10)

,al,amazing,area,bar,bathroom,bed,bien,breakfast,buena,building,c,cama,central,central park,chambre,check,clean,close,comfortable,comfy,d,da,day,del,desayuno,desk,die,e,el,emplacement,en,est,et,excelente,excellent,floor,friendly,friendly helpful,friendly staff,good,...,personal,personnel,place,posizione,price,que,r,really,restaurant,room,room clean,s,sehr,service,size,small,square,staff,staff friendly,state,station,stay,subway,super,t,tel,time,time square,todo,tr,tr s,ubicaci,ubicaci n,una,und,view,walk,y,york,zimmer
0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.320789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.306815,0.0,0.419775,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.466262,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.474326,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.585494,0.000000,0.656124,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.476129,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.424313,0.373842,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.412992,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.271285,0.0,0.0,0.0,0.000000,0.0,0.0,0.361264,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.340493,0.367445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
5,0.0,0.0,0.0,0.455833,0.000000,0.00000,0.0,0.384182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.496093,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.558403,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
6,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.454279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.370422,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.349124,0.376760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.371245,0.0,0.0
7,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
8,0.0

# 3. Data Exploration

Now that we have a td-idf weight matrix, this can be fed directly into a predictive model. Before we do this, let's explore the current data a little more:

### Merging datasets

In [14]:
# Merging td-idf weight matrix with original DataFrame
model = pd.merge(data, model, left_index=True, right_index=True)

In [15]:
# Printing the first 10 reviews left
model.head(10)

,Hotel Names,Sentiment,Comments,Filtered Review Text,Polarity,al,amazing,area,bar,bathroom,bed,bien,breakfast,buena,building,c,cama,central,central park,chambre,check,clean,close,comfortable,comfy,d,da,day,del,desayuno,desk,die,e,el,emplacement,en,est,et,excelente,excellent,...,personal,personnel,place,posizione,price,que,r,really,restaurant,room,room clean,s,sehr,service,size,small,square,staff,staff friendly,state,station,stay,subway,super,t,tel,time,time square,todo,tr,tr s,ubicaci,ubicaci n,una,und,view,walk,y,york,zimmer
0,Hyatt Place New York City/Times Square,1,perfect location! Very friendly staff that hel...,"[perfect, location, friendly, staff, helped, s...",0.368056,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.320789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.466262,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.474326,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,Hyatt Place New York City/Times Square,1,Clean and comfortable rooms,"[clean, comfortable, room]",0.383333,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.585494,0.000000,0.656124,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.476129,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,Hyatt Place New York City/Times Square,0,The best place to be,"[best, place]",1.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Hyatt Place New York City/Times Square,1,"excellent location, very comfortable rooms and...","[excellent, location, comfortable, room, close...",0.700000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.424313,0.373842,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.412992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.271285,0.0,0.0,0.0,0.000000,0.0,0.0,0.361264,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.340493,0.367445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,Hyatt Place New York City/Times Square,1,Service,[service],0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
5,Hyatt Place New York City/Times Square,1,I got to the hotel 1 hour and 30 minutes early...,"[got, hotel, hour, minute, early, fault, check...",0.083333,0.0,0.0,0.0,0.455833,0.000000,0.00000,0.0,0.384182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.496093,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.558403,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
6,Hyatt Place New York City/Times Square,1,Esta muy cerca del Time Square. Las habitacion...,"[esta, muy, cerca, del, time, square, la, habi...",0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.454279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.

In [16]:
posmodel=model[model["Sentiment"]!="0"]
posmodel.drop("Sentiment",axis=1,inplace=True)
posmodel.drop("Comments",axis=1,inplace=True)
posmodel.drop("Polarity",axis=1,inplace=True)
posmodel.drop("Filtered Review Text",axis=1,inplace=True)
posmodel

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Hotel Names,al,amazing,area,bar,bathroom,bed,bien,breakfast,buena,building,c,cama,central,central park,chambre,check,clean,close,comfortable,comfy,d,da,day,del,desayuno,desk,die,e,el,emplacement,en,est,et,excelente,excellent,floor,friendly,friendly helpful,friendly staff,...,personal,personnel,place,posizione,price,que,r,really,restaurant,room,room clean,s,sehr,service,size,small,square,staff,staff friendly,state,station,stay,subway,super,t,tel,time,time square,todo,tr,tr s,ubicaci,ubicaci n,una,und,view,walk,y,york,zimmer
0,Hyatt Place New York City/Times Square,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.320789,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.306815,0.0,0.419775,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.466262,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.474326,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,Hyatt Place New York City/Times Square,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.585494,0.000000,0.656124,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.476129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,Hyatt Place New York City/Times Square,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.424313,0.373842,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.412992,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.271285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.361264,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.340493,0.367445,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,Hyatt Place New York City/Times Square,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
5,Hyatt Place New York City/Times Square,0.0,0.0,0.0,0.455833,0.000000,0.000000,0.000000,0.384182,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.496093,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.558403,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
6,Hyatt Place New York City/Times Square,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [17]:
hotelnames=posmodel.groupby("Hotel Names")

posvecmodel=hotelnames.mean()
posvecmodel

,al,amazing,area,bar,bathroom,bed,bien,breakfast,buena,building,c,cama,central,central park,chambre,check,clean,close,comfortable,comfy,d,da,day,del,desayuno,desk,die,e,el,emplacement,en,est,et,excelente,excellent,floor,friendly,friendly helpful,friendly staff,good,...,personal,personnel,place,posizione,price,que,r,really,restaurant,room,room clean,s,sehr,service,size,small,square,staff,staff friendly,state,station,stay,subway,super,t,tel,time,time square,todo,tr,tr s,ubicaci,ubicaci n,una,und,view,walk,y,york,zimmer
Hotel Names,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1 Hotel Central Park,0.004591,0.030327,0.003611,0.006325,0.005705,0.025611,0.003558,0.017724,0.003383,0.002421,0.006738,0.002998,0.032191,0.032824,0.003337,0.005413,0.024106,0.003344,0.032434,0.006236,0.009985,0.011808,0.005708,0.002627,0.004136,0.004159,0.010804,0.030745,0.017417,0.010712,0.016857,0.008924,0.012951,0.021502,0.040444,0.001336,0.055706,0.010671,0.018497,0.023725,...,0.016527,0.004394,0.010976,0.019318,0.002053,0.004427,0.002792,0.017866,0.028387,0.046947,0.002524,0.020099,0.016853,0.033131,0.001484,0.004084,0.001896,0.089402,0.006800,0.000000,0.002202,0.010293,0.006025,0.016275,0.009532,0.007099,0.010485,0.001929,0.012846,0.009314,0.009340,0.010751,0.010783,0.002250,0.015665,0.013755,0.004414,0.016018,0.006092,0.008859
AC Hotel by Marriott New York Downtown,0.010181,0.024852,0.012882,0.004535,0.009924,0.048060,0.010363,0.014681,0.006477,0.002499,0.005679,0.003496,0.000407,0.000464,0.011296,0.007333,0.086598,0.015840,0.037746,0.019443,0.008082,0.017578,0.011446,0.007938,0.003060,0.016325,0.007288,0.031405,0.010971,0.007192,0.016586,0.003531,0.016993,0.007939,0.013194,0.010111,0.030521,0.004242,0.012770,0.036255,...,0.018142,0.006850,0.008599,0.015448,0.008216,0.003105,0.008177,0.015659,0.008572,0.074729,0.015250,0.026981,0.011936,0.018346,0.009709,0.005347,0.000405,0.078118,0.011482,0.000000,0.006865,0.021613,0.014580,0.024572,0.010763,0.007275,0.008027,0.000412,0.008596,0.008314,0.008337,0.007121,0.007143,0.005161,0.013348,0.082245,0.012249,0.020266,0.006998,0.008233
AKA Central Park,0.002868,0.013561,0.015056,0.007241,0.006256,0.022830,0.005293,0.002649,0.008347,0.001062,0.003065,0.017191,0.048893,0.052312,0.002761,0.006349,0.035628,0.012298,0.020979,0.011350,0.005625,0.010575,0.005487,0.015599,0.003224,0.005381,0.012396,0.039910,0.020469,0.002949,0.013261,0.005672,0.007736,0.012762,0.036344,0.002437,0.025676,0.015304,0.013297,0.016274,...,0.025826,0.004111,0.011989,0.010873,0.006873,0.009864,0.005581,0.010953,0.005301,0.072079,0.003779,0.011336,0.008362,0.011417,0.020556,0.005495,0.004117,0.087960,0.009679,0.000000,0.004755,0.017238,0.020292,0.008672,0.006834,0.002665,0.009757,0.004188,0.004628,0.005347,0.005362,0.032195,0.032291,0.005111,0.009544,0.001793,0.010435,0.027755,0.008254,0.000668
AKA Times Square,0.006555,0.013841,0.005191,0.002288,0.007148,0.020336,0.016399,0.003596,0.003364,0.000629,0.014694,0.007503,0.016125,0.000000,0.006439,0.003581,0.027835,0.014428,0.021995,0.009477,0.003107,0.013119,0.004521,0.013913,0.001210,0.004962,0.007470,0.062214,0.020734,0.016351,0.015159,0.010920,0.022745,0.010963,0.030509,0.002099,0.028781,0.013350,0.011000,0.028123,...,0.027055,0.018167,0.013246,0.026861,0.004044,0.011142,0.004982,0.007388,0.009349,0.041906,0.000000,0.025884,0.020363,0.006861,0.012874,0.001482,0.035419,0.064866,0.009263,0.000637,0.000859,0.013968,0.003219,0.011085,0.014236,0.002190,0.039552,0.034575,0.007165,0.017063,0.017110,0.023151,0.021976,0.004657,0.019422,0.002356,0.002964,0.019586,0.007717,0.000508
Aliz Hotel Times Square,0.007639,0.028374,0.005501,0.051634,0.013891,0.037439,0.008272,0.013071,0.003607,0.003956,0.003424,0.004218,0.007807,0.001002,0.008143,0.011811,0.074057,0.018487,0.038293,0.011735,0.007175,0.012903,0.006304,0.007734,0.001918,0.006287,0.012099,0.029012,0.009458,0.008942,0.009143,0.008464,0.014152,0.003743,0.017778,0.016184,0.027126,0.007938,0.007462,0.033905,...,0.0080

In [18]:
negmodel=model[model["Sentiment"]!="1"]
negmodel.drop("Sentiment",axis=1,inplace=True)
negmodel.drop("Comments",axis=1,inplace=True)
negmodel.drop("Polarity",axis=1,inplace=True)
negmodel.drop("Filtered Review Text",axis=1,inplace=True)
negmodel

,Hotel Names,al,amazing,area,bar,bathroom,bed,bien,breakfast,buena,building,c,cama,central,central park,chambre,check,clean,close,comfortable,comfy,d,da,day,del,desayuno,desk,die,e,el,emplacement,en,est,et,excelente,excellent,floor,friendly,friendly helpful,friendly staff,...,personal,personnel,place,posizione,price,que,r,really,restaurant,room,room clean,s,sehr,service,size,small,square,staff,staff friendly,state,station,stay,subway,super,t,tel,time,time square,todo,tr,tr s,ubicaci,ubicaci n,una,und,view,walk,y,york,zimmer
2,Hyatt Place New York City/Times Square,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
10,Hyatt Place New York City/Times Square,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
17,Hyatt Place New York City/Times Square,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
30,Hyatt Place New York City/Times Square,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
38,Hyatt Place New York City/Times Square,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
43,Hyatt Place New York City/Times Square,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.493395,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.410477,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.443669,0.000000,0.000000,0.0,0.274297,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
47,Hyatt Place New York City/Times Square,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0

In [19]:
negmodel.groupby("Hotel Names")
hotelnames=negmodel.groupby("Hotel Names")
negvecmodel=hotelnames.mean()
negvecmodel.info()
negvecmodel



<class 'pandas.core.frame.DataFrame'>
Index: 209 entries, 1 Hotel Central Park to citizenM New York Bowery
Columns: 115 entries, al to zimmer
dtypes: float64(115)
memory usage: 189.4+ KB


,al,amazing,area,bar,bathroom,bed,bien,breakfast,buena,building,c,cama,central,central park,chambre,check,clean,close,comfortable,comfy,d,da,day,del,desayuno,desk,die,e,el,emplacement,en,est,et,excelente,excellent,floor,friendly,friendly helpful,friendly staff,good,...,personal,personnel,place,posizione,price,que,r,really,restaurant,room,room clean,s,sehr,service,size,small,square,staff,staff friendly,state,station,stay,subway,super,t,tel,time,time square,todo,tr,tr s,ubicaci,ubicaci n,una,und,view,walk,y,york,zimmer
Hotel Names,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1 Hotel Central Park,0.000000,0.000000,0.000000,0.033818,0.124290,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027112,0.000000,0.006232,0.021734,0.019408,0.000000,0.000000,0.000000,0.029548,0.008026,0.008952,0.000000,0.000000,0.000000,0.009026,0.051816,0.016214,0.000000,0.007559,0.012048,0.005486,0.000000,0.013499,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.023031,0.000000,0.018379,0.023139,0.041308,0.000000,0.000000,0.186593,0.000000,0.029255,0.018004,0.023183,0.000000,0.091420,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017636,0.021039,0.000000,0.014934,0.000000,0.009482
AC Hotel by Marriott New York Downtown,0.000000,0.010330,0.000000,0.000000,0.033308,0.042069,0.000000,0.035581,0.000000,0.011925,0.000000,0.000000,0.000000,0.000000,0.000000,0.036004,0.031258,0.000000,0.000000,0.000000,0.038060,0.014971,0.067643,0.015976,0.014439,0.049766,0.000000,0.009593,0.032732,0.000000,0.029405,0.000000,0.000000,0.000000,0.009342,0.026316,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.024631,0.000000,0.000000,0.000000,0.000000,0.109681,0.000000,0.012556,0.000000,0.051413,0.023160,0.011368,0.000000,0.043352,0.000000,0.000000,0.000000,0.056136,0.000000,0.000000,0.069340,0.000000,0.021325,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.010825,0.000000,0.026316
AKA Central Park,0.023584,0.000000,0.000000,0.032979,0.000000,0.000000,0.000000,0.027795,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022322,0.012748,0.036610,0.000000,0.000000,0.069580,0.000000,0.060599,0.110029,0.000000,0.132210,0.024499,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.101117,0.000000,0.000000,0.068143,0.030813,0.000000,0.029224,0.066667,0.000000,0.041020,0.000000,0.118695,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025314,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026438,0.000000,0.066667,0.000000,0.018363,0.000000,0.000000
AKA Times Square,0.000000,0.000000,0.000000,0.083333,0.064769,0.015485,0.008489,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009037,0.000000,0.000000,0.000000,0.000000,0.000000,0.025264,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006139,0.000000,0.000000,0.000000,0.000000,0.015911,0.000000,0.000000,0.034636,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.032974,0.035831,0.000000,0.000000,0.081882,0.000000,0.027619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040170,0.009523,0.023628,0.000000,0.000000,0.008607,0.008631,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006928,0.000000,0.000000
Aliz Hotel Times Square,0.006570,0.009193,0.006925,0.048005,0.009845,0.026664,0.000000,0.005952,0.000000,0.000000,0.000000,0.000000,0.003123,0.000000,0.025917,0.014533,0.010442,0.007368,0.005652,0.000000,0.023877,0.012883,0.022745,0.000904,0.000000,0.014831,0.008751,0.039153,0.012959,0.000000,0.006716,0.009617,0.006307,0.000000,0.002110,0.024441,0.005666,0.000000,0.007752,0.008254,...,0.0030

In [20]:
mainmodel=pd.concat([posvecmodel,negvecmodel],axis=1)
mainmodel.info()


<class 'pandas.core.frame.DataFrame'>
Index: 209 entries, 1 Hotel Central Park to citizenM New York Bowery
Columns: 230 entries, al to zimmer
dtypes: float64(230)
memory usage: 377.2+ KB


In [21]:
featvect=[]
for i in range(len(mainmodel.index)):
    featvect.append(mainmodel.iloc[i].tolist())


featvect


[[0.004590546660397578,
  0.030326914590788873,
  0.0036108646509743045,
  0.006324966881164596,
  0.005704901451062345,
  0.02561086174697536,
  0.003557750864748869,
  0.01772447829933952,
  0.0033830612782538277,
  0.0024208496729652925,
  0.006737545669302368,
  0.0029984838622500115,
  0.03219141136578449,
  0.032824322879065074,
  0.003336984328827323,
  0.005412718677916639,
  0.02410638761183297,
  0.003344041456836213,
  0.032433607623113536,
  0.006236431028412304,
  0.009984686415327706,
  0.011807816230380347,
  0.0057076024823013065,
  0.002626735454487159,
  0.004136173786298142,
  0.004159418306322586,
  0.010804078682341883,
  0.030745276029357275,
  0.01741650976394556,
  0.010711995058234126,
  0.01685665176010384,
  0.008923782257413162,
  0.012950694170425707,
  0.021501874935053795,
  0.04044377826623612,
  0.0013363889703730913,
  0.05570612991950514,
  0.010671415859617542,
  0.018497231045184453,
  0.023724692564790338,
  0.004711280695143006,
  0.06302368046909

In [22]:
datas = pd.read_csv('../input/scoredata/NYscores.csv',sep=";",encoding="utf-8")
datas.info()
groupscore= datas.groupby("Hotel Names")
scores=groupscore.mean()

endmodel=pd.concat([mainmodel,scores],axis=1)
endmodel

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 8 columns):
Hotel Names        209 non-null object
Staff              209 non-null float64
Facilities         209 non-null float64
Cleanliness        209 non-null float64
Comfort            209 non-null float64
Value for money    209 non-null float64
Location           209 non-null float64
Free WiFi          209 non-null float64
dtypes: float64(7), object(1)
memory usage: 13.1+ KB


,al,amazing,area,bar,bathroom,bed,bien,breakfast,buena,building,c,cama,central,central park,chambre,check,clean,close,comfortable,comfy,d,da,day,del,desayuno,desk,die,e,el,emplacement,en,est,et,excelente,excellent,floor,friendly,friendly helpful,friendly staff,good,...,really,restaurant,room,room clean,s,sehr,service,size,small,square,staff,staff friendly,state,station,stay,subway,super,t,tel,time,time square,todo,tr,tr s,ubicaci,ubicaci n,una,und,view,walk,y,york,zimmer,Staff,Facilities,Cleanliness,Comfort,Value for money,Location,Free WiFi
Hotel Names,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1 Hotel Central Park,0.004591,0.030327,0.003611,0.006325,0.005705,0.025611,0.003558,0.017724,0.003383,0.002421,0.006738,0.002998,0.032191,0.032824,0.003337,0.005413,0.024106,0.003344,0.032434,0.006236,0.009985,0.011808,0.005708,0.002627,0.004136,0.004159,0.010804,0.030745,0.017417,0.010712,0.016857,0.008924,0.012951,0.021502,0.040444,0.001336,0.055706,0.010671,0.018497,0.023725,...,0.000000,0.000000,0.186593,0.000000,0.029255,0.018004,0.023183,0.000000,0.091420,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017636,0.021039,0.000000,0.014934,0.000000,0.009482,9.0,8.6,9.0,8.8,7.6,9.6,9.1
AC Hotel by Marriott New York Downtown,0.010181,0.024852,0.012882,0.004535,0.009924,0.048060,0.010363,0.014681,0.006477,0.002499,0.005679,0.003496,0.000407,0.000464,0.011296,0.007333,0.086598,0.015840,0.037746,0.019443,0.008082,0.017578,0.011446,0.007938,0.003060,0.016325,0.007288,0.031405,0.010971,0.007192,0.016586,0.003531,0.016993,0.007939,0.013194,0.010111,0.030521,0.004242,0.012770,0.036255,...,0.000000,0.000000,0.109681,0.000000,0.012556,0.000000,0.051413,0.023160,0.011368,0.000000,0.043352,0.000000,0.000000,0.000000,0.056136,0.000000,0.000000,0.069340,0.000000,0.021325,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.010825,0.000000,0.026316,9.0,8.8,9.1,9.0,8.5,8.9,8.4
AKA Central Park,0.002868,0.013561,0.015056,0.007241,0.006256,0.022830,0.005293,0.002649,0.008347,0.001062,0.003065,0.017191,0.048893,0.052312,0.002761,0.006349,0.035628,0.012298,0.020979,0.011350,0.005625,0.010575,0.005487,0.015599,0.003224,0.005381,0.012396,0.039910,0.020469,0.002949,0.013261,0.005672,0.007736,0.012762,0.036344,0.002437,0.025676,0.015304,0.013297,0.016274,...,0.000000,0.029224,0.066667,0.000000,0.041020,0.000000,0.118695,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025314,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026438,0.000000,0.066667,0.000000,0.018363,0.000000,0.000000,8.8,8.3,8.6,8.6,7.7,9.6,9.0
AKA Times Square,0.006555,0.013841,0.005191,0.002288,0.007148,0.020336,0.016399,0.003596,0.003364,0.000629,0.014694,0.007503,0.016125,0.000000,0.006439,0.003581,0.027835,0.014428,0.021995,0.009477,0.003107,0.013119,0.004521,0.013913,0.001210,0.004962,0.007470,0.062214,0.020734,0.016351,0.015159,0.010920,0.022745,0.010963,0.030509,0.002099,0.028781,0.013350,0.011000,0.028123,...,0.000000,0.000000,0.081882,0.000000,0.027619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040170,0.009523,0.023628,0.000000,0.000000,0.008607,0.008631,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006928,0.000000,0.000000,9.1,8.5,8.8,8.8,7.7,9.5,8.9
Aliz Hotel Times Square,0.007639,0.028374,0.005501,0.051634,0.013891,0.037439,0.008272,0.013071,0.003607,0.003956,0.003424,0.004218,0.007807,0.001002,0.008143,0.011811,0.074057,0.018487,0.038293,0.011735,0.007175,0.012903,0.006304,0.007734,0.001918,0.006287,0.012099,0.029012,0.009458,0.008942,0.009143,0.008464,0.014152,0.003743,0.017778,0.016184,0.027126,0.007938,0.007462,0.033905,...,0.016648,0.014258,0.161227,0.002728,0.024712,0.002891,0.034001,0.004033,0.106335,0.000550,0.048965,0.000000,0.000000,0.00

In [23]:
w=endmodel.iloc[:,:230]
z=endmodel.iloc[:,230:]


In [24]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
xx = sc.fit_transform(w)

from sklearn import preprocessing


min_max_scaler = preprocessing.MinMaxScaler()
w = min_max_scaler.fit_transform(xx)
print(w[:,6])


print(xx[:,6])
w_train, w_test,z_train,z_test = train_test_split(w,z,test_size=0.33, random_state=0)

[0.07698791 0.22424082 0.1145312  0.35487673 0.17900301 0.03902782
 0.3083009  0.20097602 0.15373256 0.04838394 0.05487339 0.43345061
 0.24266296 0.18639172 0.21100283 0.27291946 0.19999315 0.170941
 0.29550735 0.14315203 0.12086303 0.13267484 0.22875609 0.2457499
 0.06000948 0.18030843 0.         0.20156037 0.12898146 0.
 0.07089008 0.13808793 0.12833285 0.15246941 0.16191905 0.15312798
 0.117085   0.35669376 0.31156975 0.         0.23839896 0.26547763
 0.14866028 0.16344152 0.42784347 0.26136743 0.2078585  0.2999565
 0.19980438 0.20683078 0.17480286 0.15535264 0.04091654 0.15021922
 0.28894837 0.09812998 0.09588319 0.18929853 0.11794541 0.15074403
 0.26469291 0.26459502 0.23453738 0.1341947  0.14121249 0.21809734
 0.19538555 0.08191635 0.20166248 0.11603254 0.12517057 0.16720063
 0.04104967 0.27303307 0.28801717 0.37102201 0.22195741 0.20871647
 0.23157081 0.17430987 0.13794973 0.25230023 0.27730347 0.10707392
 0.15333129 0.09827119 0.14007632 0.14327567 0.19114491 1.
 0.14887204 0.1

In [25]:
stafflist=endmodel["Staff"].tolist()
facilitieslist=endmodel["Facilities"].tolist()
cleanlist=endmodel["Cleanliness"].tolist()
comfortlist=endmodel["Comfort"].tolist()
moneylist=endmodel["Value for money"].tolist()
locationlist=endmodel["Location"].tolist()
wifilist=endmodel["Free WiFi"].tolist()

wifilist

[9.1,
 8.4,
 9.0,
 8.9,
 8.4,
 8.7,
 8.5,
 9.1,
 9.1,
 9.4,
 9.6,
 9.0,
 8.2,
 8.6,
 8.6,
 8.1,
 8.9,
 6.6,
 8.6,
 8.1,
 8.9,
 8.8,
 9.3,
 8.5,
 8.4,
 7.8,
 8.8,
 8.5,
 8.7,
 9.5,
 5.8,
 8.9,
 7.8,
 8.1,
 7.2,
 8.8,
 8.8,
 8.4,
 8.1,
 -1.0,
 8.5,
 8.5,
 9.1,
 8.7,
 8.9,
 8.6,
 8.0,
 8.1,
 8.6,
 8.7,
 8.2,
 8.6,
 8.8,
 8.2,
 8.5,
 8.2,
 7.5,
 8.5,
 8.0,
 8.6,
 8.8,
 8.4,
 8.3,
 8.1,
 8.4,
 8.9,
 8.5,
 8.7,
 7.9,
 8.9,
 8.4,
 8.0,
 -1.0,
 8.1,
 8.1,
 8.4,
 8.2,
 8.5,
 7.9,
 8.6,
 8.8,
 8.0,
 8.1,
 7.8,
 8.5,
 9.2,
 8.9,
 9.1,
 6.7,
 8.8,
 7.4,
 8.4,
 6.4,
 8.7,
 8.8,
 8.0,
 8.2,
 9.0,
 8.8,
 8.6,
 7.1,
 8.3,
 8.3,
 8.2,
 8.5,
 8.9,
 8.7,
 7.8,
 7.8,
 7.9,
 8.1,
 6.9,
 9.4,
 8.7,
 -1.0,
 8.6,
 -1.0,
 -1.0,
 7.5,
 7.9,
 8.3,
 8.9,
 8.7,
 9.0,
 7.5,
 7.5,
 -1.0,
 -1.0,
 8.6,
 8.2,
 8.6,
 7.3,
 10.0,
 8.3,
 8.9,
 8.8,
 8.5,
 -1.0,
 8.8,
 9.2,
 9.4,
 8.8,
 8.8,
 8.7,
 8.6,
 -1.0,
 8.8,
 8.4,
 9.1,
 9.1,
 -1.0,
 7.8,
 6.2,
 8.2,
 7.9,
 -1.0,
 8.3,
 10.0,
 8.7,
 8.1,
 8.6,
 7.7,
 8.9,
 7.5,
 8.

In [26]:
from sklearn import svm
from sklearn.metrics import mean_absolute_error
from sklearn import tree


In [27]:

X1 = featvect[205:892]
y1= wifilist[205:892]
regr1 =svm.SVR()
regr1.fit(X1, y1)
pr=regr1.predict(featvect[0:205])

mean_absolute_error(wifilist[0:205], pr)

1.1309472469449802

In [28]:
X1 = featvect[205:892]
y1= locationlist[205:892]
regr1 =tree.DecisionTreeRegressor()
regr1.fit(X1, y1)
pr=regr1.predict(featvect[0:205])

mean_absolute_error(locationlist[0:205], pr)

0.5360975609756095

In [29]:
X1 = featvect[205:892]
y1= moneylist[205:892]
regr1 =tree.DecisionTreeRegressor()
regr1.fit(X1, y1)
pr=regr1.predict(featvect[0:205])

mean_absolute_error(moneylist[0:205], pr)

0.5199999999999999

In [30]:
X1 = featvect[52:230]
y1= wifilist[52:230]
regr1 =svm.SVR()
regr1.fit(X1, y1)
pr=regr1.predict(featvect[0:52])

mean_absolute_error(wifilist[0:52], pr)

0.648169650838432

In [31]:
X_train = featvect[52:54]
y_train= [5,6]
X_test=featvect[53:55]
y_test=[7,8]
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression(random_state=0)

logr.fit(X_train,y_train)


y_pred = logr.predict(X_test)
print(y_pred)
print(y_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print("Logistic")
print(cm)

[6 5]
[7, 8]
Logistic
[[0 0 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [1 0 0 0]]


In [32]:
X1 = featvect[205:892]
y1= comfortlist[205:892]
regr1 =tree.DecisionTreeRegressor()
regr1.fit(X1, y1)
pr=regr1.predict(featvect[0:205])

mean_absolute_error(comfortlist[0:205], pr)

0.7136585365853656

In [33]:
X1 = featvect[205:892]
y1= cleanlist[205:892]
regr1 =tree.DecisionTreeRegressor()
regr1.fit(X1, y1)
pr=regr1.predict(featvect[0:205])

mean_absolute_error(cleanlist[0:205], pr)

0.535121951219512

In [34]:
X1 = featvect[205:892]
y1= stafflist[205:892]
regr1 =tree.DecisionTreeRegressor()
regr1.fit(X1, y1)
pr=regr1.predict(featvect[0:205])

mean_absolute_error(stafflist[0:205], pr)

0.538048780487805

In [35]:
X1 = featvect[49:209]
y1= comfortlist[49:209]
regr1 =tree.DecisionTreeRegressor()
regr1.fit(X1, y1)
pr=regr1.predict(featvect[0:49])

mean_absolute_error(comfortlist[0:49], pr)

0.3326530612244898